In [2]:
from sqlmodel import Field, SQLModel, create_engine
from typing import Optional
from datetime import datetime

class Membres(SQLModel, table = True):
    id : int | None = Field(default = None, primary_key = True)
    nom : str 
    email : str
    carte_acces_id: Optional[int]= Field(default = None, foreign_key= Cartes_acces.id)

class Cartes_acces(SQLModel, table = True):
    id : int | None = Field(default = None, primary_key = True)
    numero_unique: Optional[int]= Field(default = None, foreign_key= Membres.id)

NameError: name 'Cartes_acces' is not defined

In [ ]:


sqlite_file_name = "database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=True)

SQLModel.metadata.create_all(engine)


In [3]:
from sqlmodel import SQLModel, Session, create_engine, select, Relationship
import datetime
from random import choice, randint
from faker import Faker
from typing import List

# Générateur de données fictives
fake = Faker()

# Créer une base de données SQLite en mémoire (changez l'URL pour une autre base si nécessaire)
engine = create_engine("sqlite:///:memory:")

# Définir les modèles (tu peux réutiliser ceux que tu as déjà définis)

class Cartes_acces(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    numero_unique: int | None
    membres: List['Membres'] = Relationship(back_populates="carte_acces")

class Membres(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    nom: str
    email: str
    carte_acces_id: Optional[int] = Field(default=None, foreign_key='cartes_acces.id')
    carte_acces: "Cartes_acces" = Relationship(back_populates="membres")

class Coachs(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    nom_coach: str
    specialite: str

class Cours(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    nom_cours: str
    horaire: datetime.datetime = Field(default=None)
    capacite_max: int
    coach_id: Optional[int] = Field(default=None, foreign_key='coachs.id')

class Inscription(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    date_inscription: datetime.datetime = Field(default_factory=datetime.datetime.utcnow)
    membre_id: Optional[int] = Field(default=None, foreign_key='membres.id')
    cours_id: Optional[int] = Field(default=None, foreign_key='cours.id')

# Créer les tables dans la base de données
SQLModel.metadata.create_all(engine)

# Générer des données pour les cartes d'accès et les membres
def generate_cartes_acces(session: Session):
    for i in range(1, 13):  # 12 cartes d'accès pour les membres 1 à 12
        carte = Cartes_acces(numero_unique=randint(100000, 999999))
        session.add(carte)
    session.commit()

def generate_membres(session: Session):
    for i in range(1, 13):  # 12 membres
        membre = Membres(
            nom=fake.name(),
            email=fake.email(),
            carte_acces_id=i  # Assigner les cartes d'accès 1 à 12
        )
        session.add(membre)
    session.commit()

def generate_coachs(session: Session):
    coach_names = ['Alice', 'Bob', 'Charlie']
    specialites = ['Yoga', 'Musculation', 'Pilates']
    
    for i in range(3):  # 3 coachs
        coach = Coachs(
            nom_coach=coach_names[i],
            specialite=specialites[i]
        )
        session.add(coach)
    session.commit()

def generate_cours(session: Session):
    jours_semaine = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    
    for i in range(1, 21):  # 20 cours
        jour = choice(jours_semaine)
        horaire = datetime.datetime(2024, 12, randint(1, 31), randint(8, 18), randint(0, 59))
        capacite_max = 8
        coach_id = randint(1, 3)  # Assignation aléatoire d'un coach (1 à 3)
        
        cours = Cours(
            nom_cours=f"{fake.word().capitalize()} {jour}",
            horaire=horaire,
            capacite_max=capacite_max,
            coach_id=coach_id
        )
        session.add(cours)
    session.commit()

def generate_inscriptions(session: Session):
    for i in range(1, 21):  # Inscrire 20 membres à 20 cours
        membre_id = randint(1, 12)  # Choisir un membre aléatoire
        cours_id = randint(1, 20)  # Choisir un cours aléatoire
        
        inscription = Inscription(
            membre_id=membre_id,
            cours_id=cours_id
        )
        session.add(inscription)
    session.commit()

# Insérer les données dans la base
with Session(engine) as session:
    # Générer les cartes d'accès et les membres
    generate_cartes_acces(session)
    generate_membres(session)
    
    # Générer les coachs
    generate_coachs(session)
    
    # Générer les cours
    generate_cours(session)
    
    # Générer les inscriptions
    generate_inscriptions(session)

    # Vérification des données insérées
    membres = session.query(Membres).all()
    for membre in membres:
        print(f"Membre ID: {membre.id}, Nom: {membre.nom}, Email: {membre.email}")
    
    cours = session.query(Cours).all()
    for cours in cours:
        print(f"Cours ID: {cours.id}, Nom: {cours.nom_cours}, Horaire: {cours.horaire}, Coach ID: {cours.coach_id}")
    
    inscriptions = session.query(Inscription).all()
    for inscription in inscriptions:
        print(f"Inscription ID: {inscription.id}, Membre ID: {inscription.membre_id}, Cours ID: {inscription.cours_id}")


Membre ID: 1, Nom: Thomas Medina, Email: mayojoshua@example.org
Membre ID: 2, Nom: Dr. Nathan Smith, Email: rodriguezashley@example.net
Membre ID: 3, Nom: Monica Banks, Email: eric07@example.com
Membre ID: 4, Nom: Amanda Olson, Email: ortizsarah@example.org
Membre ID: 5, Nom: Dr. Melanie Evans, Email: bradymegan@example.org
Membre ID: 6, Nom: Cheyenne Holder, Email: rbush@example.com
Membre ID: 7, Nom: Casey Harris, Email: anna71@example.net
Membre ID: 8, Nom: Selena Lyons, Email: carmencarrillo@example.net
Membre ID: 9, Nom: Jeffrey Foster, Email: wsmith@example.com
Membre ID: 10, Nom: Richard Hartman, Email: bward@example.org
Membre ID: 11, Nom: Ryan Chavez, Email: steve04@example.net
Membre ID: 12, Nom: Joshua Baker, Email: hmccall@example.net
Cours ID: 1, Nom: Organization Mardi, Horaire: 2024-12-12 11:20:00, Coach ID: 2
Cours ID: 2, Nom: Similar Lundi, Horaire: 2024-12-31 15:01:00, Coach ID: 1
Cours ID: 3, Nom: Seem Dimanche, Horaire: 2024-12-28 08:10:00, Coach ID: 2
Cours ID: 4, 

/tmp/ipykernel_13521/2041430623.py:123: DeprecationWarning: 
        🚨 You probably want to use `session.exec()` instead of `session.query()`.

        `session.exec()` is SQLModel's own short version with increased type
        annotations.

        Or otherwise you might want to use `session.execute()` instead of
        `session.query()`.
        
  membres = session.query(Membres).all()
/tmp/ipykernel_13521/2041430623.py:127: DeprecationWarning: 
        🚨 You probably want to use `session.exec()` instead of `session.query()`.

        `session.exec()` is SQLModel's own short version with increased type
        annotations.

        Or otherwise you might want to use `session.execute()` instead of
        `session.query()`.
        
  cours = session.query(Cours).all()
/tmp/ipykernel_13521/2041430623.py:131: DeprecationWarning: 
        🚨 You probably want to use `session.exec()` instead of `session.query()`.

        `session.exec()` is SQLModel's own short version with increased 